# Assignment 3
## [Intelligent Systems 2018-2](https://fagonzalezo.github.io/is-2018-2/)

## Markov Decision Processes and Reinforcement Learning
### Deadline: *Thursday December 13th*

**Names: Cristian Alejandro Pulido Quintero**




b 70## 1. The Optimal Parking Problem

In the optimal parking problem (OPP), the goal is to find a parking spot that is as close as possible to a store so that the driver minimizes the distance that he/she has to walk. Parking spots are numbered $1,\dots,N$. $1$ corresponds to the closest spot to the store and $N$ to the farthest. 

![parking image](parking.jpg)

The driver starts at the front of the $N$-th parking spot. At the front of each parking spot $i$ the driver can execute two actions `park` or `continue`. The driver does not know in advance which parking spots are free or not. The probability of any parking spot to be free is $p$. When the action `park` is executed, if the parking spot is free the car will be parked at the $i$-th parking spot and this will generate a reward of $-i$ (equivalent to a cost of $i$) and the process will end. If the $i$-th parking spot is occupied and the `park` action is executed or if just the `continue` action is executed, the car will move forward towards the $i-1$ parking spot. This does not generate any reward. If the driver arrives to the parking spot $1$ and cannot park, the process will end generating a reward of $-M$ (equivalent to a cost of $M$).

### 1.1 (0.5) OPP Markov decision problem

Model the OPP as a Markov decision problem (MDP). Draw the MDP as a graph showing clearly the states, the transitions with the corresponding probabilities and rewards. 

### 1.2 (0.5) Expected reward

 
Build a function to calculate the expected reward at each parking spot for a particular instance of the OPP.

![title](P.png)

In [1]:
def expected_rewards(N, M, p):
    '''
    N: number of parking spots
    M: cost of not being able to park (positive value)
    p: probability of a parking spot being free
    policy: a list of actions to execute at each step. 
            'P' indicates 'park'. 'C' indicates 'continue'
            
    Returns: a dictionary with the expected rewards at each parking spot.
    '''
    result = {i:0 for i in range(1,N+1)}
    ### your code here
    result[1]=max([-p-M*(1-p),-M])
    for i in range(2,N+1):
        result[i]=max([-i*p+(1-p)*result[i-1],result[i-1]])

    return result


### 1.3 (0.5) Policy evaluation

Build a function to calculate the expected reward at each parking spot for a particular instance of the OPP and for a given policy. The policy is specified by indicating which action to take at each parking spot.

In [2]:
def eval_policy(N, M, p, policy):
    '''
    N: number of parking spots
    M: cost of not being able to park (positive value)
    p: probability of a parking spot being free
    policy: a dictionary with the corresponding actions for each parking spot.
            'P' indicates 'park', 'C' indicates 'continue'. For instance for N = 3
            this could be a posible policy:
               {1:'P', 2:'P', 3:'C'}
            
    Returns: a dictionary with the expected rewards at each parking spot.
    '''
    result = {i:0 for i in range(1,N+1)}
    ### your code here
    if policy[1] == 'P':
        result[1] = -p-(1-p)*M
    else:
        result[1]= -M
    for i in range(2,N+1):
        if policy[i] == 'P':
            result[i] = -i*p+(1-p)*result[i-1]
        elif policy[i] == 'C':
            result[i]= result[i-1]
    return result

### 1.4 (0.5) Optimal policy

Build a function to calculate the policy that maximizes the reward at each parking spot for a particular instance of the OPP. 

In [3]:
def optimal_policy(N, M, p):
    '''
    N: number of parking spots
    M: cost of not being able to park (positive value)
    p: probability of a parking spot being free
            
    Returns: a dictionary with the actions of the optimal policy for each parking spot.
            'P' indicates 'park', 'C' indicates 'continue'.
    '''
    result = {i:'C' for i in range(1,N+1)}
    ### your code here
    
    for k in range(10):
        eval_p=eval_policy(N,M,p,result)
        if -p-(1-p)*M > -M:
            result[1]='P'
        else:
            result[1]='C'
        for i in range(2,N+1):

            if -i*p+(1-p)*eval_p[i-1] > eval_p[i-1]:
                result[i]='P'
            else:
                result[i]='C'
    
    return result


### 1.5 (0.5) Minimum cost

If the cost of not parking, $M$, is small it may make sense to execute `continue` at the farthest parking spots. On the other hand, if the cost of not parking is large enough, it may be good idea to park as soon as possible, i.e., to execute park in all the parking spots.  Find a formula in terms of $N$ and $p$ for the minimum value of $M$ such that the optimal policy is to execute `park` at all parking spots. Clearly explain the derivation of the formula.

La idea principal para hallar la formula se obtiene de la relacion en la que la recompensa esperada por ejecutar en
cada espacio la accion de parquear sea mayor que cualquier otra, como en el problema para cada estado solo se pueden ejecutar 2 acciones comparamos la accion de parquear y continuar, por tanto debemos hallar el $M$ que cumpla la relacion
$$E[s|P]>E[s|C]$$
lo cual es equivalente a 
$$-\sum_{i=1}^N{ip(1-p)^{N-i}}-m(1-p)^N > -\sum_{i=1}^{N-1}{ip(1-p)^{N-1-i}}-M(1-p)^{N-1}$$
bajo propiedades de las sumatorias
$$p(1-p)^{N-1}+\sum_{i=2}^N{ip(1-p)^{N-1}}-\sum_{i=2}^N{(i-1)p(1-p)^{N-1}}<M[(1-p)^{N-1}-(1-p)^N]$$ 

$$p(1-p)^{N-1}+\sum_{i=2}^N{p(1-p)^{N-1}}<M[(1-p)^{N-1}-(1-p)^N]$$
note que en la ultima sumatoria las potencias estan en el rango de $2$ a $N-1$ y junto con el termino de la izquierda 
la sumatoria queda
$$p\sum_{i=1}^{N}{(1-p)^{i-1}}<Mp(1-p)^{N-1}$$
por la sumatoria geometrica
$$\frac{1-(1-p)^N}{1-(1-p)}<M(1-p)^{N-1}$$
Por tanto se obtiene la relacion
$$M>\frac{1-(1-p)^N}{p(1-p)^{N-1}}$$
Que es equivalente a 
$$M>\sum_{i=1}^{N}{\frac{1}{(1-p)^{i-1}}}$$


Si $N=1$ se obtiene $M > 1$ sin importar la probabilidad

ejemplos:

In [79]:
for p in range(1,10):
    print("p="+str(p/10.0),"M=1",optimal_policy(1,1,p/10.0),"M=1.0001",optimal_policy(1,1.0001,p/10.0))

p=0.1 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.2 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.3 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.4 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.5 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.6 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.7 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.8 M=1 {1: 'C'} M=1.0001 {1: 'P'}
p=0.9 M=1 {1: 'C'} M=1.0001 {1: 'P'}


Si $N=2$ se obtiene $$M>1+\frac{1}{1-p}=\frac{2-p}{1-p}$$

In [80]:
for i in range(1,10):
    p=i/10.0
    M = (2-p)/(1-p)
    print("p="+str(p),"M="+str(M),optimal_policy(2,M,p),"M="+str(M+.0001),optimal_policy(2,M+0.0001,p))

p=0.1 M=2.111111111111111 {1: 'P', 2: 'C'} M=2.1112111111111114 {1: 'P', 2: 'P'}
p=0.2 M=2.25 {1: 'P', 2: 'C'} M=2.2501 {1: 'P', 2: 'P'}
p=0.3 M=2.428571428571429 {1: 'P', 2: 'C'} M=2.428671428571429 {1: 'P', 2: 'P'}
p=0.4 M=2.666666666666667 {1: 'P', 2: 'C'} M=2.666766666666667 {1: 'P', 2: 'P'}
p=0.5 M=3.0 {1: 'P', 2: 'C'} M=3.0001 {1: 'P', 2: 'P'}
p=0.6 M=3.4999999999999996 {1: 'P', 2: 'C'} M=3.5000999999999998 {1: 'P', 2: 'P'}
p=0.7 M=4.333333333333333 {1: 'P', 2: 'C'} M=4.333433333333333 {1: 'P', 2: 'P'}
p=0.8 M=6.000000000000001 {1: 'P', 2: 'C'} M=6.000100000000001 {1: 'P', 2: 'P'}
p=0.9 M=11.000000000000004 {1: 'P', 2: 'C'} M=11.000100000000003 {1: 'P', 2: 'P'}


Si $N=3$ se obtiene $$M>1+\frac{1}{1-p}+\frac{1}{(1-p)^2}$$

In [99]:
for i in range(1,10):
    p=i/10.0
    M = 1+1/(1-p)+1/(1-p)**2
    print("p="+str(p),"M="+str(M),optimal_policy(3,M,p),"M="+str(M+.0001),optimal_policy(3,M+0.0001,p))

p=0.1 M=3.3456790123456788 {1: 'P', 2: 'P', 3: 'C'} M=3.345779012345679 {1: 'P', 2: 'P', 3: 'P'}
p=0.2 M=3.8125 {1: 'P', 2: 'P', 3: 'C'} M=3.8126 {1: 'P', 2: 'P', 3: 'P'}
p=0.3 M=4.469387755102042 {1: 'P', 2: 'P', 3: 'P'} M=4.469487755102041 {1: 'P', 2: 'P', 3: 'P'}
p=0.4 M=5.444444444444445 {1: 'P', 2: 'P', 3: 'C'} M=5.444544444444444 {1: 'P', 2: 'P', 3: 'P'}
p=0.5 M=7.0 {1: 'P', 2: 'P', 3: 'C'} M=7.0001 {1: 'P', 2: 'P', 3: 'P'}
p=0.6 M=9.75 {1: 'P', 2: 'P', 3: 'C'} M=9.7501 {1: 'P', 2: 'P', 3: 'P'}
p=0.7 M=15.444444444444443 {1: 'P', 2: 'P', 3: 'C'} M=15.444544444444443 {1: 'P', 2: 'P', 3: 'P'}
p=0.8 M=31.000000000000014 {1: 'P', 2: 'P', 3: 'C'} M=31.000100000000014 {1: 'P', 2: 'P', 3: 'P'}
p=0.9 M=111.00000000000006 {1: 'P', 2: 'P', 3: 'C'} M=111.00010000000006 {1: 'P', 2: 'P', 3: 'P'}


Para $N=6$

In [101]:
for i in range(1,10):
    p=i/10.0
    M = 1+1/(1-p)+1/(1-p)**2+1/(1-p)**3+1/(1-p)**4+1/(1-p)**5
    print("p="+str(p),"M="+str(M),optimal_policy(6,M,p),"M="+str(M+.0001),optimal_policy(6,M+0.0001,p))

p=0.1 M=7.935087808430286 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=7.935187808430285 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.2 M=11.2587890625 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=11.2588890625 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.3 M=17.49967275539954 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=17.49977275539954 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.4 M=30.65020576131688 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=30.65030576131688 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.5 M=63.0 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=63.0001 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.6 M=162.09374999999994 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=162.09384999999995 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.7 M=587.4609053497938 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'C'} M=587.4610053497938 {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P'}
p=0.8 M=3906.00000000

## 2. Learning to play Snake

The objective is to develop a reinforcement learning algorithm that learns to play the Snake game.

<img src="https://cloud.githubusercontent.com/assets/2750531/24808769/cc825424-1bc5-11e7-816f-7320f7bda2cf.gif" alt="Snake snapshot" width="320"/>

We will use as a basis the code in this [project](https://github.com/YuriyGuts/snake-ai-reinforcement) from [Yuriy Guts](https://github.com/YuriyGuts). The first step would be to clone that project and start looking at the code. This project builds a Deep Q-learning agent that learns to play the Snake game. It is important to notice that we do not want to reproduce this part of the project, but rather use it as a basis for building two simpler agents: one based on pure Q-learning and another one based on approximated Q-learning. 


### 2.1 (1.0) Q-learning agent

Implement a Q-learning agent that learns to play Snake. During learning, the agent must keep a table for the Q function, i.e. a table that keeps track of different combinations of states and actions. Here a state corresponds to a posible configuration of the environment. Because of these, we will need to work with small environments. The suggestion is to create smaller environments ($5 \times 5$ or $6 \times 6$) following the examples in the [levels folder](https://github.com/YuriyGuts/snake-ai-reinforcement/tree/master/snakeai/levels).

You have to create a new class that extends from the `AgentBase` class:


Para el codigo de esta clase modifique el archivo original de train para que se pueda hacer llamado a esta clase, la cual al hacer el entrenamiento crea un diccionario Q donde las keys son los estados por los que ha pasado y los valores tambien son diccionarios de la forma {0:a,1:b,2:c,3:c} que representa la recompensa esperada por cada accion 0: frente 1 : izquierda 2: derecha y el valor de 3 son las veces que pasa por el estado durante el entrenamiento. Este diccionario se guarda como un archivo "q.npy" el cual despues es cargado al jugar para escoger la mejor accion.

La accion que toma es la que maximiza los valores de Q

In [ ]:
import collections
from snakeai.agent import AgentBase
from snakeai.gameplay.entities import ALL_SNAKE_ACTIONS
import numpy as np
class QAgent(AgentBase):
    """ Represents an intelligent agent for the Snake environment. """

    def __init__(self, num_last_frames=1):
        """
        Create a new DQN-based agent.
        
        Args:
            model: a compiled DQN model.
            num_last_frames (int): the number of last frames the agent will consider.
            memory_size (int): memory size limit for experience replay (-1 for unlimited). 
        """
        
        self.num_last_frames = num_last_frames
        self.frames = None
    
    def begin_episode(self):
        """ Reset the agent for a new episode. """
        self.frames = None

    def act(self, observation, reward):
        """
        Choose the next action to take.
        Args:
            observation: observable state for the current timestep. 
            reward: reward received at the beginning of the current timestep.
        Returns:
            The index of the action to take next.
        """
        Q = np.load('q.npy').item()
        s=tuple([tuple(row) for row in observation])
        if s in Q:
            m=np.argmax([Q[s][0],Q[s][1],Q[s][2]])
            return m
        else:
            return int(3*np.random.rand())

    def end_episode(self):
        """ Notify the agent that the episode has ended. """
        pass

    def get_last_frames(self, observation):
        """
        Get the pixels of the last `num_last_frames` observations, the current frame being the last.
        
        Args:
            observation: observation at the current timestep. 

        Returns:
            Observations for the last `num_last_frames` frames.
        """
        frame = observation
        if self.frames is None:
            self.frames = collections.deque([frame] * self.num_last_frames)
        else:
            self.frames.append(frame)
            self.frames.popleft()
        return np.expand_dims(self.frames, 0)
    
    def train(self, env, num_episodes=1000, discount_factor=0.9, alpha=0.05, k=10):
        """
        Train the agent to perform well in the given Snake environment using Q learning.
        
        Args:
            env:
                an instance of Snake environment.
            num_episodes (int):
                the number of episodes to run during the training.
            discount_factor (float):
                discount factor (gamma) for computing the value function.
            alpha (float): 
                moving average parameter.
            k (int): 
                exploration function parameter.
        """
        Q={}
        for episode in range(num_episodes):
            # Reset the environment for the new episode.
            timestep = env.new_episode()

            self.begin_episode()
            game_over = False
            loss = 0.0

            # Observe the initial state.
            state = timestep.observation
            Q[tuple([tuple(row) for row in state])]={0:0,1:0,2:0,3:1}


            while not game_over:          
                
                ran= np.random.rand()
                s=tuple([tuple(row) for row in state])
                if ran > 0.8:    
                    action = np.random.randint(env.num_actions)
                else:
                    action = np.argmax([Q[s][0],Q[s][1],Q[s][2]])


                # Act on the environment.
                env.choose_action(action)
                timestep = env.timestep()

                reward = timestep.reward
                new_state=tuple([tuple(row) for row in timestep.observation])
                if new_state not in Q:
                    Q[new_state]={0:0,1:0,2:0,3:1}
                    Q[tuple([tuple(row) for row in state])][action]=(1-alpha)*Q[tuple([tuple(row) for row in state])][action]+alpha*reward
                else:
                    f1=Q[new_state][0]+k*1.0/Q[new_state][3]
                    f2=Q[new_state][1]+k*1.0/Q[new_state][3]
                    f3=Q[new_state][2]+k*1.0/Q[new_state][3]

                    max_Q_new_state=discount_factor*max(f1,f2,f3)
                    sample=reward+max_Q_new_state
                    Q[tuple([tuple(row) for row in state])][action]=(1-alpha)*Q[tuple([tuple(row) for row in state])][action]+alpha*sample
                    Q[new_state][3]+=1

                    
                   
                # Remember a new piece of experience.
                state_next = timestep.observation
                game_over = timestep.is_episode_end
                state = state_next
        np.save('q.npy', Q) 

Ejemplo tabla valores de Q despues de un entrenamiento con 10000 juegos consecutivos, con dimensiones 6x6 y longitud inicial 2

In [19]:
import numpy as np
Q=np.load("q-10000.npy")
Q.any()

{((4, 4, 4, 4, 4, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 1, 2, 0, 0, 4),
  (4, 0, 3, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 4, 4, 4, 4, 4)): {0: 0.3777022998576814, 1: 0, 2: 0, 3: 1},
 ((4, 4, 4, 4, 4, 4),
  (4, 0, 2, 0, 0, 4),
  (4, 1, 3, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 4, 4, 4, 4, 4)): {0: -0.9797287034615834,
  1: 8.389707589765731,
  2: 4.8292318244495185,
  3: 6363},
 ((4, 4, 4, 4, 4, 4),
  (4, 2, 3, 0, 0, 4),
  (4, 1, 0, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 4, 4, 4, 4, 4)): {0: -0.9777198937391993,
  1: 9.237070057799995,
  2: -0.9752620736568646,
  3: 5827},
 ((4, 4, 4, 4, 4, 4),
  (4, 3, 3, 0, 0, 4),
  (4, 2, 1, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 4, 4, 4, 4, 4)): {0: 2.5678993256460987,
  1: 6.964825326896091,
  2: 0.10810373129387009,
  3: 461},
 ((4, 4, 4, 4, 4, 4),
  (4, 3, 0, 0, 0, 4),
  (4, 3, 1, 0, 0, 4),
  (4, 2, 0, 0, 0, 4),
  (4, 0, 0, 0, 0, 4),
  (4, 4, 4, 4, 4, 4)): {0: 4.9560491088710945,
  1: 2.901

Como se puede ver cada estado esta representado por una matriz de 6x6 donde cada valor representa un objeto en el juego 0:vacio 1: manzana 2: cabeza 3:cuerpo 4: pared, la cual es el identificador en el diccionario y sus valores son las recompensas esperadas de cada accion junto con la catidad de veces que se ha pasado por ese estado, como se dijo anteriormente. 

The class will include additional methods, in particular methods for training and housekeeping of internal variables. You can take a look to the `DeepQNetworkAgent` [class](https://github.com/YuriyGuts/snake-ai-reinforcement/blob/master/snakeai/agent/dqn.py) to see how to interact with the environment during training.  You should report the results of the training process and include a detailed analysis of these results. Also, you should be able to save the parameters of a trained agent and load them in order to show how it performs in different environments. Look at the functionality of the project that allows to load pretrained agents.

### 2.2 (1.0) Approximate Q-learning agent

Implement an approximate Q-learning agent that learns to play Snake. The agent must calculate the $Q$ function as a linear combination of features extracted from the environment:

$$ Q(s,a) = w_1f_1(s,a)+w_2f_2(s,a)\dots w_nf_n(s,a)$$

The agent must calculate these features based on the current state of the board which can be accessed through the observations from the environment. You should try different features and discuss them.

You have to create a new class that extends from the `AgentBase` class:

Para el codigo de esta clase modifique el archivo original de train para que se pueda hacer llamado a esta clase, la cual al hacer el entrenamiento crea un vector de pesos $w$ por cada accion, en total 3 vectores, los cuales aproximan los valores $Q$ de recompensa esperada para cada accion, estos vectores se guardan como un archivo "w.npy" el cual despues es cargado al jugar para escoger la mejor accion.

La accion que toma es la que maximiza los valores de Q, que se aproximan para cada accion con la funcion lineal descrita por los pesos w.


Los parametros escogidos como funciones de un estado fueron.

- tamano de la serpiente
- distancia euclidiana de la cabeza a la comida
- distancia manhattan de la cabeza a la comida
- distancia en x de la cabeza a la comida
- distancia en y de la cabeza a la comida
- verificar si la cabeza esta en una esquina
- distancia a cada una de las paredes
- numero de casillas vacias


In [ ]:
from snakeai.agent import AgentBase
from snakeai.gameplay.entities import ALL_SNAKE_ACTIONS
import numpy as np
import math
class ApproximateQAgent(AgentBase):
    """ Represents an intelligent agent for the Snake environment. """

    def __init__(self, num_last_frames=1):
        """
        Create a new DQN-based agent.
        
        Args:
            model: a compiled DQN model.
            num_last_frames (int): the number of last frames the agent will consider.
            memory_size (int): memory size limit for experience replay (-1 for unlimited). 
        """
        
        self.num_last_frames = num_last_frames
        self.frames = None
    
    def begin_episode(self):
        """ Reset the agent for a new episode. """
        self.frames = None

    def act(self, observation, reward):
        """
        Choose the next action to take.
        Args:
            observation: observable state for the current timestep. 
            reward: reward received at the beginning of the current timestep.
        Returns:
            The index of the action to take next.
        """
        w = np.load('w.npy').item()
        a0=np.dot(w[0],self.features(observation))  
        a1=np.dot(w[1],self.features(observation))  
        a2=np.dot(w[2],self.features(observation))  

        return np.argmax([a0,a1,a2])     
                
        

    def end_episode(self):
        """ Notify the agent that the episode has ended. """
        pass

    def features(self, observation):
        """
        Calculate features from an observation of the environment.
        
        Args:
            observation: observable state for the current timestep. 
        
        Returns:
            A list/array of feature values.
        """

        
        #tamaño
        size=1+len(np.where(observation==3)[0])
        #cabeza 
        cabeza=(np.where(observation==2)[0][0],np.where(observation==2)[1][0])
        #comida
        comida=(np.where(observation==1)[0][0],np.where(observation==2)[1][0])
        #distancia euclidiana cabeza - comida
        de=math.sqrt((cabeza[0]-comida[0])**2+(cabeza[1]-comida[1])**2)
        #distancia taxista cabeza - comida
        dt=abs(cabeza[0]-comida[0])+abs(cabeza[1]-comida[1])
        # cabeza - comida x
        x=abs(cabeza[0]-comida[0])
        # cabeza - comida y
        y=abs(cabeza[1]-comida[1])
        # cabeza en esquina
        ce = cabeza == (1,1) or cabeza == (1,5) or cabeza == (5,1) or cabeza == (5,5)
        # distancia pared norte
        pn = cabeza[1]
        # distancia pared sur
        ps = 6-cabeza[1]
        # distancia pared este
        pe = 6-cabeza[0]
        # distancia pared oeste
        po = cabeza[0]
        #vacios
        v=len(np.where(observation==0)[0])
        return [de,dt,v,int(ce),size,x,y,pn,ps,pe,po]


    def train(self, env, num_episodes=1000, discount_factor=0.9, alpha=0.05, k=10):
        """
        Train the agent to perform well in the given Snake environment using Q learning.
        
        Args:
            env:
                an instance of Snake environment.
            num_episodes (int):
                the number of episodes to run during the training.
            discount_factor (float):
                discount factor (gamma) for computing the value function.
            alpha (float): 
                moving average parameter.
            k (int): 
                exploration function parameter.
        """

        w0=np.zeros(11)
        w1=np.zeros(11)
        w2=np.zeros(11)
        w={0:w0,1:w1,2:w2}
        for episode in range(num_episodes):
            # Reset the environment for the new episode.
            timestep = env.new_episode()

            self.begin_episode()
            game_over = False
            loss = 0.0

            # Observe the initial state.
            state = timestep.observation

            
            
            while not game_over:    
                ran= np.random.rand()
                if ran > 0.8:    
                    action = np.random.randint(env.num_actions)
                else:
                    feat=self.features(state)
                    a0=np.dot(w[0],feat)  
                    a1=np.dot(w[1],feat)  
                    a2=np.dot(w[2],feat)  
        
                    action=np.argmax([a0,a1,a2])     
                

                                   

                # Act on the environment.
                env.choose_action(action)
                timestep = env.timestep()

                reward = timestep.reward

                new_state=timestep.observation
                feat=self.features(new_state)
                a0=np.dot(w[0],feat)  
                a1=np.dot(w[1],feat)  
                a2=np.dot(w[2],feat)  
    
                Q_max=max(a0,a1,a2)
                diff=reward+discount_factor*Q_max-np.dot(w[action],self.features(state))
                w[action]+=alpha*diff*np.array(self.features(state))
                   
                   
                # Remember a new piece of experience.
                state_next = timestep.observation
                game_over = timestep.is_episode_end
                state = state_next
        np.save('w.npy', w) 

The class will include additional methods, in particular methods for training and housekeeping of internal variables. You can take a look to the `DeepQNetworkAgent` [class](https://github.com/YuriyGuts/snake-ai-reinforcement/blob/master/snakeai/agent/dqn.py) to see how to interact with the environment during training.  You should report the results of the training process and include a detailed analysis of these results. Also, you should be able to save the parameters of a trained agent and load them in order to show how it performs in different environments. Look at the functionality of the project that allows to load pretrained agents.

### 2.3 (0.5) Evaluating the performance of different learning methods

Make an analysis of the performance of the different learning agents: Q-lerning, Approximate Q-learnging and Deep Reinforcement Learning (using the pretrained models available in the [release page](https://github.com/YuriyGuts/snake-ai-reinforcement/releases)). The performance of the models must be measured in terms of the average maximum length of the snake.

Para hacer las comparaciones lo primero que se hizo fue pre-entrenar un modelo de Deep Reinforcement Learning para las dimensiones usadas (6x6) en principio con 3000 juegos, pero cuando estaba entrenando completo un juego llenando todas las casillas despues de 2400 intentos, por tanto no se completo, pero con el modelo resultante se corrieron 10 jugadas con los siguientes resultados

In [32]:
import pandas as pd 
data = pd.read_csv("dqn.csv") 
data.head(10)

,action_counter_0,action_counter_1,action_counter_2,fruits_eaten,mean_reward,sum_episode_rewards,termination_reason,timesteps_survived
0,15,11,15,8,1.243902,51,hit_wall,41
1,31,20,13,8,0.796875,51,hit_wall,64
2,27,9,21,8,0.894737,51,hit_wall,57
3,16,11,11,6,0.842105,32,hit_own_body,38
4,19,15,14,7,0.854167,41,hit_own_body,48
5,5,5,3,5,1.846154,24,hit_wall,13
6,30,12,24,7,0.621212,41,hit_wall,66
7,12,11,14,7,1.108108,41,hit_own_body,37
8,27,13,15,6,0.581818,32,hit_wall,55
9,17,5,18,9,1.550000,62,hit_wall,40


A partir de las frutas comidas se puede inferir la longitud final, recordando que empiza con longitud 2, por lo tanto de los 10 juegos tomamos 2 valores:
      - promedio : 9.1
      - maximo :   11

Para el primer punto con el QAgent primero se entreno con 2400 intentos para comparar con el anterior, con los siguientes resultados

In [33]:
data = pd.read_csv("q-2400.csv") 
data.head(10)

,action_counter_0,action_counter_1,action_counter_2,fruits_eaten,mean_reward,sum_episode_rewards,termination_reason,timesteps_survived
0,5,3,1,2,0.666667,6,hit_wall,9
1,2,1,1,0,-0.250000,-1,hit_wall,4
2,2,0,0,1,1.000000,2,hit_wall,2
3,10,5,0,4,1.133333,17,hit_wall,15
4,4,2,1,1,0.285714,2,hit_wall,7
5,3,4,2,2,0.666667,6,hit_wall,9
6,4,2,0,2,1.000000,6,hit_wall,6
7,7,10,2,2,0.315789,6,hit_wall,19
8,7,4,1,0,-0.083333,-1,hit_wall,12
9,5,3,1,1,0.222222,2,hit_wall,9


      - promedio : 3.5
      - maximo :   6

Despues se entreno con 10000 intentos 

In [35]:
data = pd.read_csv("q-10000.csv") 
data.head(10)

,action_counter_0,action_counter_1,action_counter_2,fruits_eaten,mean_reward,sum_episode_rewards,termination_reason,timesteps_survived
0,24,16,0,9,1.550000,62,hit_own_body,40
1,13,4,7,4,0.708333,17,hit_wall,24
2,18,12,0,9,2.066667,62,hit_own_body,30
3,10,13,1,6,1.333333,32,hit_own_body,24
4,35,24,1,6,0.533333,32,hit_wall,60
5,7,6,1,3,0.785714,11,hit_wall,14
6,5,1,5,3,1.000000,11,hit_wall,11
7,9,4,3,4,1.062500,17,hit_wall,16
8,13,0,7,5,1.200000,24,hit_wall,20
9,36,27,1,8,0.796875,51,hit_wall,64


      - promedio : 7.7
      - maximo :   11
      
Como se puede ver mejoro la politica de acciones al entrenarlo por mas tiempo, pasando de 3.5 a 7.7 en longitud promedio, pero sigue siendo inferior al primer agente el cual necesito menos tiempo de entrenamiento, ademas para este ultimo como se guarda un diccionario con los estados y los valores de Q para cada accion, mientras mas tiempo entrena aunque aproxima mejor los valores de Q reales necesita mayor memoria para guardar la tabla de estados y acciones.

Para el Agente ApproximateQAgent aun falta exploracion ya que los intentos que se han realizado no muestran valores optimos, por ejemplo, usando todas las varibles dichas anteriormente, junto con 3000 juegos de entrenamiento se obtienen los siguientes resultados

In [36]:
data = pd.read_csv("aq-3000.csv") 
data.head(10)

,action_counter_0,action_counter_1,action_counter_2,fruits_eaten,mean_reward,sum_episode_rewards,termination_reason,timesteps_survived
0,2,0,0,0,-0.5,-1,hit_wall,2
1,2,0,0,0,-0.5,-1,hit_wall,2
2,2,0,0,0,-0.5,-1,hit_wall,2
3,2,0,0,0,-0.5,-1,hit_wall,2
4,2,0,0,0,-0.5,-1,hit_wall,2
5,2,0,0,0,-0.5,-1,hit_wall,2
6,2,0,0,0,-0.5,-1,hit_wall,2
7,2,0,0,0,-0.5,-1,hit_wall,2
8,2,0,0,0,-0.5,-1,hit_wall,2
9,2,0,0,0,-0.5,-1,hit_wall,2


## Submission

The assignment must be submitted as a compressed file containing this notebook as well as all supporting files through the following Dropbox [file request](https://www.dropbox.com/request/ctSp1nsjXNcQKBXVDXfu), before midnight of the deadline date. The file must be named as is-assign3-unalusername1-unalusername2-unalusername3.zip, where unalusername is the user name assigned by the university (include the usernames of all the members of the group).

### Test functions

You can use the following functions to test your solutions.

In [158]:
import hashlib

def round_dict(dictionary, digits):
    return {key:round(value, digits) for key, value in dictionary.items()}

def hash_list(lista):
    string = str(lista)
    hash_object = hashlib.sha1(string.encode('latin_1'))
    hex_dig = hash_object.hexdigest()
    return hex_dig

def hash_dict(dictionary):
    string = str([(key,dictionary[key]) 
                  for key in sorted(dictionary.keys())])
    hash_object = hashlib.sha1(string.encode('latin_1'))
    hex_dig = hash_object.hexdigest()
    return hex_dig

def compare(vals1, vals2, error):
    for i, val in enumerate(vals1):
        if abs(vals2[i] - val) > error:
            return False
    return True

def compare_dicts(d1, d2, error):
    if d1.keys() != d2.keys():
        return False
    for i in d1:
        if abs(d1[i] - d2[i]) > error:
            print(d1[i],d2[i])
            return False
    return True

def compare_dicts_d(d1, d2, error):
    if d1.keys() != d2.keys():
        return False
    for i in d1:
        if d1[i] != d2[i]:
            return False
    return True

def rand_policy(N, seed):
    import random
    random.seed(seed)
    policy = random.choices(['P', 'C'], k=N)
    return {i:policy[i-1] for i in range(1, N + 1)}

def test1():
    # case 1
    N, M, p = 10, 20, 0.05
    answer = {1: -19.05, 2: -18.1975, 3: -17.437625, 4: -16.765744, 5: -16.177457, 6: -15.668584, 7: -15.235155, 8: -14.873397, 9: -14.579727, 10: -14.350741}
    result = expected_rewards(N, M, p)
    if not compare_dicts(answer, result, 0.001):
        print("test 1 fail 1")
        return False
    # case 2
    N, M, p = 10, 100, 0.5
    answer = {1: -50.5, 2: -26.25, 3: -14.625, 4: -9.3125, 5: -7.15625, 6: -6.578125, 7: -6.578125, 8: -6.578125, 9: -6.578125, 10: -6.578125}
    result = expected_rewards(N, M, p)
    if not compare_dicts(answer, result, 0.001):
        print("test 1 fail 2")
        return False

    # case 3
    N, M, p = 50, 10000, 0.1
    answer = "02047eaf324745e70fd84be50c79b8303e2d8b34"
    result = hash_dict(round_dict(expected_rewards(N, M, p), 4))
    if not answer == result:
        print("test 1 fail 3")
        print(answer, result)
        return False
    
    # case 4
    N, M, p = 100, 200, 0.05
    answer = "a6d6934abea3631e95484057c40d28638d2bf51b"
    result = hash_dict(round_dict(expected_rewards(N, M, p), 4))
    if not answer == result:
        print("test 1 fail 4")
        print(answer, result)
        return False

    return True


def test2():
    # case 1
    N, M, p = 10, 20, 0.05
    pol = rand_policy(N, 124)
    answer = {1: -20.0, 2: -20.0, 3: -19.15, 4: -19.15, 5: -18.4425, 6: -18.4425, 7: -17.870375, 8: -17.376856, 9: -17.376856, 10: -17.376856}
    result = eval_policy(N, M, p, pol)
    if not compare_dicts(answer, result, 0.001):
        return False
    
    # case 2
    N, M, p = 10, 100, 0.5
    pol = rand_policy(N, 123)
    answer = {1: -50.5, 2: -26.25, 3: -14.625, 4: -9.3125, 5: -9.3125, 6: -7.65625, 7: -7.65625, 8: -7.828125, 9: -7.828125, 10: -8.914062}
    result = eval_policy(N, M, p, pol)
    if not compare_dicts(answer, result, 0.001):
        return False

    # case 3
    N, M, p = 50, 10000, 0.1
    pol = rand_policy(N, 125)
    answer = "184ea94fb88be2b8b64362b8a1bcc564d9f06acb"
    result = hash_dict( round_dict(eval_policy(N, M, p, pol), 4))
    if not answer == result:
        print("test 2 fail 3")
        return False

    # case 4
    N, M, p = 100, 200, 0.05
    pol = rand_policy(N, 126)
    answer =  "b08946cb4b27ce82526f68c1c02b0b6ad6ab50a8"
    result = hash_dict( round_dict(eval_policy(N, M, p, pol), 4))
    if not answer == result:
        print("test 2 fail 4")
        return False
    
    return True



def test3():
    # case 1
    N, M, p = 10, 20, 0.05
    answer = {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P', 7: 'P', 8: 'P', 9: 'P', 10: 'P'}
    result = optimal_policy(N, M, p)
    if not compare_dicts_d(answer, result, 0.001):
        print("test 3 fail 1")
        return False

    # case 2
    N, M, p = 10, 100, 0.5
    answer = {1: 'P', 2: 'P', 3: 'P', 4: 'P', 5: 'P', 6: 'P', 7: 'C', 8: 'C', 9: 'C', 10: 'C'}
    result = optimal_policy(N, M, p)
    if not compare_dicts_d(answer, result, 0.001):
        print("test 3 fail 2")
        return False

    # case 3
    N, M, p = 50, 10000, 0.1
    answer = "2057cfda7eb7b171be964137648d85a6b4f4314f"
    result = hash_dict(optimal_policy(N, M, p))
    if not answer == result:
        print("test 3 fail 3")
        return False

    # case 4
    N, M, p = 100, 200, 0.05
    answer = "f707b5fdbd0f29a4550b4730c7594aa61b373e17"
    result = hash_dict(optimal_policy(N, M, p))
    if not answer == result:
        print("test 3 fail 4")
        return False

    return True

def evaluate():
    score = 0 
    for test in [test1, test2, test3]:
        if test():
            score += 1
    return score

print ("Score: ", evaluate(), "/ 3")

test 2 fail 3
Score:  2 / 3
